## SRJ

## Adapt - GCIM

## First step - To create Generating functions

## Generating fns code ready.  single & 2 singles

## Gradients added and state is ready

In [32]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
  


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    excitationlist.extend(singles)
    print('Excitations are', excitationlist)
    #print('Going to print the single generating fns.')
    for excitation in excitationlist:
        @qml.qnode(dev)
        def superposition1():
            #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
            qml.StatePrep(hf_state, wires=range(qubits))
            #print('Single Exc going in', excitation)
            qml.SingleExcitation(theta, wires=excitation)
            return qml.state()
        super1 = superposition1()
        generatingfns.append(super1)
        #print('S-Generating functions are', super1)
    #print('After adding single gfs, len is', len(generatingfns))
    print('\n\n\n')
    #print('Printing 2s generating fns')
    for i, j in itertools.combinations(excitationlist, 2):
        if len(i) == 2 and len(j) == 2:  # Ensure both are singles
            #print(f'Applying excitations: {i} and {j}')
            @qml.qnode(dev)
            def superposition2():
                #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
                qml.StatePrep(hf_state, wires=range(qubits))
                qml.SingleExcitation(theta, wires=i)
                qml.SingleExcitation(theta, wires=j)
                return qml.state()                
            super2= superposition2()
            generatingfns.append(super2)
            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]


    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                #print('Print, if this is activated - HF state')
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy, Ham_matrix


In [33]:
ash_excitation, states,eig,gs_energy,Ham_matrix = adaptvqe(adapt_it=1, e_th=1e-12)

HF state is -1.3325688880668918
Excitations are [[0, 4], [0, 6], [1, 5], [1, 7], [2, 4], [2, 6], [3, 5], [3, 7]]




The adapt iteration now is 0


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.31004515885821066 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
ash_excitation is [[2, 3, 6, 7]]
The length of ash_excitation before generating matrix is 1
0: ─╭|Ψ⟩───────────┤  State
1: ─├|Ψ⟩───────────┤  State
2: ─├|Ψ⟩─╭G²(0.79)─┤  State
3: ─├|Ψ⟩─├G²(0.79)─┤  State
4: ─├|Ψ⟩─│─────────┤  State
5: ─├|Ψ⟩─│─────────┤  State
6: ─├|Ψ⟩─├G²(0.79)─┤  State
7: ─╰|Ψ⟩─╰G²(0.79)─┤  State
State count after Generating function is {'11000011': 164, '11110000': 836}
After adding new state, the length of states is 2
Shape of Hamiltonian matrix is (256, 256)
The value of i and j is 0 0
The value of i and j is 0 1
The value of i and j is 1 0
The value of i and j is 1 1
S matrix with 1 and 1 is [[1.        +0.j 0.92387953-0.j]
 [0.92387953+0.j 1.        +0.j]]
Hamiltonian Matrix with 1 1 is [[-1.33256889+0.j -1.29045769-0.j]
 [-1.29045769+0.j -1.42037613+0.j]]
Eigenvalues are [-1.43008367 -1.0861242 ]
Ground state energy is [-1.430083670535

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


## Case 1 structure

In [16]:
dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev_meas)
def measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.counts()

dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def energy(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.expval(H)


In [17]:
# Just adding 1 double excitation and checking the energy
ash_excitation = [[2,3,6,7]]
#ash_excitation = [[0,3,5,6]]
theta = 
hf_state = np.array([1,1,1,1,0,0,0,0])
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def new_state_double(hf_state, ash_excitation, qubits):
    qml.BasisState(hf_state, wires=range(qubits))
    print(f'The value of theta is {theta} and ash_excitation is {ash_excitation[0]}')
    #qml.DoubleExcitation(theta, wires=ash_excitation[0])
    i=0
    qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))

    return qml.state()



ostate_double = new_state_double(hf_state, ash_excitation, qubits)
case1_E = energy(ostate_double)
print('Energy of the state with double excitation is', case1_E)

counts_double = measure(ostate_double)
print('State count after Generating function is', counts_double)

The value of theta is 0.7853981633974483 and ash_excitation is [2, 3, 6, 7]
Energy of the state with double excitation is -1.420376125683398
State count after Generating function is {'11000011': 151, '11110000': 849}


## Case 2: Exponential of the operator

In [19]:
# Exponential of the operator 

from pennylane.fermi import from_string
electrons = 4
qubits = 8
singles, doubles = qchem.excitations(electrons, qubits)

hf_state = np.array([1,1,1,1,0,0,0,0])  # Hartree-Fock state for 4 electrons in 8 orbitals


#print(singles_fermi)
doubles_fermi = []
excitation = [[2,3,6,7]]
#excitation = [[0,3,5,6]]  # Example double excitation indices
for ex in excitation:
    print('The ex2 is', ex)
    doubles_fermi.append(from_string(f"{ex[3]}+ {ex[2]}+ {ex[1]}- {ex[0]}-")
                       - from_string(f"{ex[0]}+ {ex[1]}+ {ex[2]}- {ex[3]}-"))
#print(doubles_fermi)


doubles_pauli = []
for op in doubles_fermi:
    print('Exc is ',op)
    doubles_pauli.append(qml.jordan_wigner(op, qubits))

#print(doubles_pauli[0])
theta = np.pi/4
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H) 

hf_energy = circuit(hf_state, electrons, qubits, H)
print('Hartree-Fock energy is', hf_energy)  

@qml.qnode(qml.device("default.qubit", wires=qubits))
def summa_trial(hf_state, doubles_pauli, theta):    
    qml.BasisState(hf_state, wires=range(qubits))
    for i in range(len(doubles_pauli)):
        print('The value of i is', i)
        qml.exp((theta/2 * doubles_pauli[i]).operation()), range(qubits)
    return qml.state()

fstate = summa_trial(hf_state, doubles_pauli, theta)

@qml.qnode(dev)
def energy_measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.expval(H)

finalenergy = energy_measure(fstate)
print('The final energy after generating function is', finalenergy)

dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev_meas)
def measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.counts()
#print('The value of energy is',summa_trial(hf_state, doubles_pauli, theta))
counts = measure(fstate)
print('State count after Generating function is', counts)

#for i, excitation in enumerate(doubles_pauli):
#k = qml.exp((doubles_pauli[0] * theta).operation())
#print('The exponent is', k)
#exp_matrix = qml.matrix(k, wire_order=range(qubits))
#j = (Ham_matrix @ exp_matrix)

#print(j)

The ex2 is [2, 3, 6, 7]
Exc is  1.0 * a⁺(7) a⁺(6) a(3) a(2)
+ -1.0 * a⁺(2) a⁺(3) a(6) a(7)
Hartree-Fock energy is -1.3325688880668922
The value of i is 0
The final energy after generating function is -1.420376125683403
State count after Generating function is {'11000011': 118, '11110000': 882}


In [6]:
#ash_excitation_45 = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [0, 3, 5, 6]]
#ash_excitation_90 = [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [0, 3, 5, 6], [0, 3, 5, 6]]

## Creating a function to generate teh exponential of the operator

## working code below


In [51]:
# Exponential of the operator 
np.set_printoptions(threshold=np.inf)

from pennylane.fermi import from_string
electrons = 4
qubits = 8
singles, doubles = qchem.excitations(electrons, qubits)

hf_state = np.array([1,1,1,1,0,0,0,0])  # Hartree-Fock state for 4 electrons in 8 orbitals


#print(singles_fermi)
doubles_fermi = []
excitation = [[2,3,6,7]]  # Example double excitation indices
for ex in excitation:
    print('The ex2 is', ex)
    doubles_fermi.append(from_string(f"{ex[3]}+ {ex[2]}+ {ex[1]}- {ex[0]}-")
                       - from_string(f"{ex[0]}+ {ex[1]}+ {ex[2]}- {ex[3]}-"))
#print(doubles_fermi)


doubles_pauli = []
for op in doubles_fermi:
    print('Exc is ',op)
    doubles_pauli.append(qml.jordan_wigner(op, qubits))

#print(doubles_pauli[0])
theta = np.pi/4


@qml.qnode(qml.device("default.qubit", wires=qubits))
def summa_trial(hf_state, doubles_pauli, theta):    
    qml.BasisState(hf_state, wires=range(qubits))
    for i in range(len(doubles_pauli)):
        #print('The value of i is', i)
        qml.exp((theta/2 * doubles_pauli[i]).operation()), range(qubits)
    return qml.state()

fstate = summa_trial(hf_state, doubles_pauli, theta)

@qml.qnode(dev)
def energy_measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.expval(H)

finalenergy = energy_measure(fstate)
print('The final energy after generating function is', finalenergy)

dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev_meas)
def measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.counts()
#print('The value of energy is',summa_trial(hf_state, doubles_pauli, theta))
counts = measure(fstate)
print('State count after Generating function is', counts)

#for i, excitation in enumerate(doubles_pauli):
print("\n")
print('Going to multiply wiht Hmat')

#k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()), range(qubits) #Original statement
k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()) #Removed the range(qubits) part
print('k1:',k1)
print('This below code is to verify the exponential of the operator')
print('\n')
print('The exponential of operator is wokring correct. Checked it wiht case 1')
devd = qml.device("default.qubit", wires=qubits)
@qml.qnode(devd)
def energyk():
    qml.BasisState(hf_state, wires=range(qubits))
    qml.exp((theta/2 * doubles_pauli[0]).operation()), range(qubits)
    return qml.expval(H)
energy1 = energyk()
print('eenergy1:', energy1)


exp_matrix1 = qml.matrix(k1, wire_order=range(qubits))
print('exp matrix1', exp_matrix1.shape)
#print('Ham matrix', Ham_matrix)
j = (Ham_matrix @ exp_matrix1)
#print('j matrix is', j)
print(j.shape)
are_equal = np.array_equal(j, Ham_matrix)
print("Matrices are exactly equal:", are_equal)

are_close = np.allclose(Ham_matrix, j, atol=1e-1)
print("Matrices are numerically close:", are_close)


The ex2 is [2, 3, 6, 7]
Exc is  1.0 * a⁺(7) a⁺(6) a(3) a(2)
+ -1.0 * a⁺(2) a⁺(3) a(6) a(7)
The final energy after generating function is -1.420376125683403
State count after Generating function is {'11000011': 150, '11110000': 850}


Going to multiply wiht Hmat
k1: Exp(1 0.04908738521234052j * (X(2) @ X(3) @ Y(6) @ X(7)) + -0.04908738521234052j * (Y(2) @ Y(3) @ Y(6) @ X(7)) + -0.04908738521234052j * (Y(2) @ X(3) @ X(6) @ X(7)) + -0.04908738521234052j * (X(2) @ Y(3) @ X(6) @ X(7)) + 0.04908738521234052j * (Y(2) @ X(3) @ Y(6) @ Y(7)) + 0.04908738521234052j * (X(2) @ Y(3) @ Y(6) @ Y(7)) + 0.04908738521234052j * (X(2) @ X(3) @ X(6) @ Y(7)) + -0.04908738521234052j * (Y(2) @ Y(3) @ X(6) @ Y(7)))
This below code is to verify the exponential of the operator


The exponential of operator is wokring correct. Checked it wiht case 1
eenergy1: -1.4203761256834058
exp matrix1 (256, 256)
(256, 256)
Matrices are exactly equal: False
Matrices are numerically close: False


## Exponential operator - fn creator

In [ ]:
from pennylane.fermi import from_string

dev = qml.device("lightning.qubit", wires= qubits)
@qml.qnode(dev)
def operator(ash_excitation[i]):
    singles_fermi = []
    doubles_fermi = []
    doubles_pauli = []
    singles_pauli = []
    if len(ash_excitation[i]) == 4:
        doubles_fermi.append(from_string(f"{ash_excitation[i][3]}+ {ash_excitation[i][2]}+ {ash_excitation[i][1]}- {ash_excitation[i][0]}-")
                       - from_string(f"{ash_excitation[i][0]}+ {ash_excitation[i][1]}+ {ash_excitation[i][2]}- {ash_excitation[i][3]}-"))
        doubles_pauli.append(qml.jordan_wigner(doubles_fermi[-1], qubits))
        k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()) #Removed the range(qubits) part
    elif len(ash_excitation[i]) == 2:
        singles_fermi.append(from_string(f"{ash_excitation[i][1]}+ {ash_excitation[i][0]}-")
                       - from_string(f"{ash_excitation[i][0]}+ {ash_excitation[i][1]}-"))
        singles_pauli.append(qml.jordan_wigner(singles_fermi[-1], qubits))
        k1 = qml.exp((theta/2 * singles_pauli[0]).operation()) #Removed the range(qubits) part
    exp_matrix1 = qml.matrix(k1, wire_order=range(qubits))
    print('exp matrix1', exp_matrix1.shape)
    return exp_matrix1


#print(doubles_pauli[0])
theta = np.pi/4


@qml.qnode(qml.device("default.qubit", wires=qubits))
def summa_trial(hf_state, doubles_pauli, theta):    
    qml.BasisState(hf_state, wires=range(qubits))
    for i in range(len(doubles_pauli)):
        #print('The value of i is', i)
        qml.exp((theta/2 * doubles_pauli[i]).operation()), range(qubits)
    return qml.state()

fstate = summa_trial(hf_state, doubles_pauli, theta)

@qml.qnode(dev)
def energy_measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.expval(H)

finalenergy = energy_measure(fstate)
print('The final energy after generating function is', finalenergy)

dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev_meas)
def measure(fstate):
    qml.StatePrep(fstate, wires=range(qubits))
    return qml.counts()
#print('The value of energy is',summa_trial(hf_state, doubles_pauli, theta))
counts = measure(fstate)
print('State count after Generating function is', counts)

#for i, excitation in enumerate(doubles_pauli):
print("\n")
print('Going to multiply wiht Hmat')

#k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()), range(qubits) #Original statement
k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()) #Removed the range(qubits) part
print('k1:',k1)
print('This below code is to verify the exponential of the operator')
print('\n')
print('The exponential of operator is wokring correct. Checked it wiht case 1')
devd = qml.device("default.qubit", wires=qubits)
@qml.qnode(devd)
def energyk():
    qml.BasisState(hf_state, wires=range(qubits))
    qml.exp((theta/2 * doubles_pauli[0]).operation()), range(qubits)
    return qml.expval(H)
energy1 = energyk()
print('eenergy1:', energy1)


exp_matrix1 = qml.matrix(k1, wire_order=range(qubits))
print('exp matrix1', exp_matrix1.shape)
#print('Ham matrix', Ham_matrix)
j = (Ham_matrix @ exp_matrix1)
#print('j matrix is', j)
print(j.shape)
are_equal = np.array_equal(j, Ham_matrix)
print("Matrices are exactly equal:", are_equal)

are_close = np.allclose(Ham_matrix, j, atol=1e-1)
print("Matrices are numerically close:", are_close)
